In [1]:
import edge_tts
import asyncio
import util    
import json
import sys
from IPython.display import Audio
from speakers.processors import load_preprocess,RvcProcessorData
from speakers.common.utils import get_abs_path
from omegaconf import OmegaConf

2023-08-24 00:12:01 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX


from speakers.common.registry import registry rvc_speakers


/media/gpt4-pdf-chatbot-langchain/pyenv-rvc-speakers/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


from speakers.common.registry import registry vits_to_voice
from speakers.common.registry import registry bark_to_voice
register_bootstrap runner_bootstrap_web
from speakers.common.registry import registry vits_voice_task
from speakers.common.registry import registry bark_voice_task
 Key with which the item will be registered {'device': 'cuda:0'}
 Key with which the item will be registered {'device': 'cuda:0', 'is_half': True}
 Key with which the item will be registered {'device': 'cuda:0', 'is_half': True, 'x_pad': 3}
 Key with which the item will be registered {'device': 'cuda:0', 'is_half': True, 'x_pad': 3, 'x_query': 10}
 Key with which the item will be registered {'device': 'cuda:0', 'is_half': True, 'x_pad': 3, 'x_query': 10, 'x_center': 60}
 Key with which the item will be registered {'device': 'cuda:0', 'is_half': True, 'x_pad': 3, 'x_query': 10, 'x_center': 60, 'x_max': 65}


In [2]:
## 初始化 处理器

speakers_config_file='speakers.yaml'
config = OmegaConf.load(get_abs_path(speakers_config_file))
print(config)
load_preprocess(config=config.get('preprocess'))
 

{'preprocess': [{'vits_processor': {'name': 'vits_to_voice', 'vits_model_path': 'model/G_953000.pth', 'voice_config_file': 'model/config.json'}}, {'rvc_processor': {'name': 'rvc_speakers', 'hubert_model_path': 'model/hubert_base.pt', 'rvc_config_file': 'rvc.yaml'}}, {'bark_processor': {'name': 'bark_to_voice', 'tokenizer_path': '/media/checkpoint/bark/bert-base-multilingual-cased', 'text_model_path': '/media/checkpoint/bark/suno/bark_v0/text_2.pt', 'coarse_model_path': '/media/checkpoint/bark/suno/bark_v0/coarse_2.pt', 'fine_model_path': '/media/checkpoint/bark/suno/bark_v0/fine_2.pt'}}], 'tasks': [{'vits_voice_task': {'name': 'vits_voice_task', 'preprocess': [{'vits': {'processor': 'vits_processor', 'processor_name': 'VITS'}}, {'rvc': {'processor': 'rvc_processor', 'processor_name': 'RVC'}}]}}, {'bark_voice_task': {'name': 'bark_voice_task', 'preprocess': [{'bark': {'processor': 'bark_processor', 'processor_name': 'BARK'}}, {'rvc': {'processor': 'rvc_processor', 'processor_name': 'RVC

In [3]:
from speakers.tasks import get_task,load_task
 

# 加载任务实例
speakers_config_file='speakers.yaml'
config = OmegaConf.load(get_abs_path(speakers_config_file))
print(config)

load_task(config.get("tasks"))

{'preprocess': [{'vits_processor': {'name': 'vits_to_voice', 'vits_model_path': 'model/G_953000.pth', 'voice_config_file': 'model/config.json'}}, {'rvc_processor': {'name': 'rvc_speakers', 'hubert_model_path': 'model/hubert_base.pt', 'rvc_config_file': 'rvc.yaml'}}, {'bark_processor': {'name': 'bark_to_voice', 'tokenizer_path': '/media/checkpoint/bark/bert-base-multilingual-cased', 'text_model_path': '/media/checkpoint/bark/suno/bark_v0/text_2.pt', 'coarse_model_path': '/media/checkpoint/bark/suno/bark_v0/coarse_2.pt', 'fine_model_path': '/media/checkpoint/bark/suno/bark_v0/fine_2.pt'}}], 'tasks': [{'vits_voice_task': {'name': 'vits_voice_task', 'preprocess': [{'vits': {'processor': 'vits_processor', 'processor_name': 'VITS'}}, {'rvc': {'processor': 'rvc_processor', 'processor_name': 'RVC'}}]}}, {'bark_voice_task': {'name': 'bark_voice_task', 'preprocess': [{'bark': {'processor': 'bark_processor', 'processor_name': 'BARK'}}, {'rvc': {'processor': 'rvc_processor', 'processor_name': 'RVC

In [5]:
voice_task = get_task("vits_voice_task")

In [6]:
from speakers.processors import get_processors,VitsProcessorData
from speakers.tasks import VitsVoiceFlowData, Runner

from collections import deque

vits_voice = get_processors('vits_processor')

rvc_processors = get_processors('rvc_processor')

# 创建双向链表
linked_list = deque()
linked_list.append('VITS')
linked_list.append('RVC') 


#noise_scale_w: noise_scale_w(控制音素发音长度)
noise_scale_w=1
#noise_scale(控制感情变化程度)
noise_scale = 0.5
#length_scale(控制整体语速)
speed = 1

# 创建一个 VitsProcessorData 实例
vits_processor_data = VitsProcessorData(
    text='我是你的情感导师，青春的时间转瞬即逝，秋水于海又岂有一只浮游',
    language=vits_voice.lang.index('简体中文'),
    speaker_id=0,
    noise_scale=noise_scale,
    speed=speed,
    noise_scale_w=noise_scale_w
)



loaded_models=[
        '%s - %s' % (
            m['metadata'].get('source', 'Unknown'),
            m['metadata'].get('name')
        )
        for m in rvc_processors.loaded_models
]
 


#变调(整数, 半音数量, 升八度12降八度-12)
f0_up_key=-1
f0_method =["pm", "harvest", "crepe", "rmvpe"]

#检索特征占比
index_rate = 0.9
#>=3则使用对harvest音高识别的结果使用中值滤波，数值为滤波半径，使用可以削弱哑音
filter_radius=1
#输入源音量包络替换输出音量包络融合比例，越靠近1越使用输出包络
rms_mix_rate=1
#后处理重采样至最终采样率，0为不进行重采样
resample_rate=0

rvc_processor_data = RvcProcessorData(  
    model_index= 0,
    f0_up_key= f0_up_key,
    f0_method= f0_method[3],
    index_rate= index_rate,
    filter_radius= filter_radius,
    rms_mix_rate= rms_mix_rate,
    resample_sr= resample_rate 
)


# 创建一个 VitsVoiceFlowData 实例，并将 VitsProcessorData 实例作为参数传递
voice_flow_data = VitsVoiceFlowData(vits=vits_processor_data,
                               rvc=rvc_processor_data)

# 创建 Runner 实例并传递上面创建的 VitsVoiceFlowData 实例作为参数
runner = Runner(
    task_id="1",
    flow_data=voice_flow_data,
    processor_q=linked_list
)
 



In [9]:
 
rate,output_audio = await voice_task.dispatch(runner=runner)


Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.446 seconds.
Prefix dict has been built successfully.


loading rmvpe model


In [10]:
from scipy.io.wavfile import write as write_wav
from IPython.display import Audio
print(rate)
# save audio to disk
write_wav(f"test.wav", rate, output_audio) 

# play text in notebook
Audio(output_audio, rate=rate)

48000
